In [1]:
class Asteroid:
    def __init__(self,ID):
        self.ID = ID

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import spiceypy as spy
from astroquery.jplhorizons import Horizons
from urllib import request
import json
from collections import OrderedDict as odict
from astropy.time import Time

In [2]:
spy.furnsh('naif0012.tls')

In [21]:
def calculate_surrogates(ID, n=1, date='2000-01-01 00:00:00'):
    if type(ID) is not str:
        raise ValueError(f"ID should be a string (even more an asteroid name), you passed {ID}")
    if type(date) is not str:
        raise ValueError(f"date should be a string as YYYY-MM-DD HH:MM:SS , you passed {date}")
        
    html=request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={ID}&cov=mat")
    json_data=json.loads(html.read().decode())
    
    rad=180/np.pi
    deg=1/rad
    AU=149597870.693 #km 
    mu=132712440023.310 #km^3/s^2
    
    Cov=np.array(json_data["orbit"]["covariance"]["data"],dtype=float)
    Cov_label=json_data["orbit"]["covariance"]["labels"]
    t=float(json_data["orbit"]["epoch"])
    nlen=len(json_data["orbit"]["elements"])
    
    elnames=[]
    elements=odict()
    for i in range(nlen):
        element=json_data["orbit"]["elements"][i]
        elements[element["name"]]=odict()
        for prop in element.keys():
            try:
                elements[element["name"]][prop]=float(element[prop])
            except:
                pass

    for elname in elements.keys():
        element=elements[elname]
        means=[elements['e']['value'],elements['q']['value'],elements['tp']['value'],elements['om']['value'],elements['w']['value'],elements['i']['value']]
        
    data=np.random.multivariate_normal(means,Cov,n, check_valid='ignore')
    
    
    e=data[:,0];q=data[:,1];tp=data[:,2];node=data[:,3];peri=data[:,4];inc=data[:,5]

    t0=float(json_data["orbit"]["epoch"])
    et0=spy.unitim(t0,"JDTDB","ET")

    a=(q/(1-e))*AU
    n=np.sqrt(mu/a**3)
    tps=np.array([spy.unitim(t,"JDTDB","ET") for t in tp])
    M=n*(et0-tps)
    Ms=np.mod(M,2*np.pi)

    Time=f'{date} UTC'
    et=spy.str2et(Time)
    
    Ast=[]
    for i in range(len(e)): 
        Ast.append(spy.conics([q[i]*AU,e[i],inc[i]*deg,node[i]*deg,peri[i]*deg,Ms[i],et0,mu],et)/AU)
        Ast[i][3::] = Ast[i][3::]*86400
    Ast=np.array(Ast)
    return Ast

In [81]:
B=calculate_surrogates('2021eu', 1, '2024-02-27 07:40:03')
B

array([[-1.07111683,  0.26440212,  0.01047198, -0.01580146, -0.01192828,
         0.00113914]])

In [27]:
ID="1"
type(ID)

str

In [103]:
class Asteroid:
    
    def __init__(self,ID):
        self.ID = ID
        if type(ID) is not str:
            raise ValueError(f"ID should be a string (even more an asteroid name), you passed {self.ID}")
            
    #CALCULATE SURROGATES POSITIONS
    def calculate_surrogates_position(self, n=1, date='2000-01-01 00:00:00'):
        if type(date) is not str:
            raise ValueError(f"date should be a string as YYYY-MM-DD HH:MM:SS , you passed {date}")
        
        html=request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
        json_data=json.loads(html.read().decode())
    
        rad=180/np.pi
        deg=1/rad
        AU=149597870.693 #km 
        mu=132712440023.310 #km^3/s^2
    
        Cov=np.array(json_data["orbit"]["covariance"]["data"],dtype=float)
        Cov_label=json_data["orbit"]["covariance"]["labels"]
        t=float(json_data["orbit"]["epoch"])
        nlen=len(json_data["orbit"]["elements"])
    
        elnames=[]
        elements=odict()
        for i in range(nlen):
            element=json_data["orbit"]["elements"][i]
            elements[element["name"]]=odict()
            for prop in element.keys():
                try:
                    elements[element["name"]][prop]=float(element[prop])
                except:
                    pass

        for elname in elements.keys():
            element=elements[elname]
            means=[elements['e']['value'],elements['q']['value'],elements['tp']['value'],elements['om']['value'],elements['w']['value'],elements['i']['value']]
        
        data=np.random.multivariate_normal(means,Cov,n, check_valid='ignore')
    
    
        e=data[:,0];q=data[:,1];tp=data[:,2];node=data[:,3];peri=data[:,4];inc=data[:,5]

        t0=float(json_data["orbit"]["epoch"])
        et0=spy.unitim(t0,"JDTDB","ET")

        a=(q/(1-e))*AU
        n=np.sqrt(mu/a**3)
        tps=np.array([spy.unitim(t,"JDTDB","ET") for t in tp])
        M=n*(et0-tps)
        Ms=np.mod(M,2*np.pi)

        Time=f'{date} UTC'
        et=spy.str2et(Time)
    
        Ast=[]
        for i in range(len(e)): 
            Ast.append(spy.conics([q[i]*AU,e[i],inc[i]*deg,node[i]*deg,peri[i]*deg,Ms[i],et0,mu],et)/AU)
            Ast[i][3::] = Ast[i][3::]*86400
        Ast=np.array(Ast)
        return Ast

In [104]:
PHA = Asteroid('2008jl3')
PHA.calculate_surrogates_position(n=3, date='2027-04-02')

array([[-9.93502621e-01, -7.14404933e-03,  9.96447817e-03,
         2.25023883e-03, -2.12886085e-02, -2.75773138e-04],
       [-9.94423075e-01,  1.80110215e-03,  1.00816722e-02,
         2.12478738e-03, -2.12896684e-02, -2.74562817e-04],
       [-9.98584259e-01,  5.15847651e-02,  1.07239499e-02,
         1.43123807e-03, -2.12751641e-02, -2.67587754e-04]])

# Prueba de get_covariance

In [107]:
#GET COVARIANCE MATRIX
class Asteroid:
    def __init__(self,ID):
        self.ID = ID
        if type(ID) is not str:
            raise ValueError(f"ID should be a string, specifically an asteroid name (EX: 2021EU), you passed {self.ID}")
            
    def get_covariance(self): 
        html=request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
        json_data=json.loads(html.read().decode())
        
        Cov=np.array(json_data["orbit"]["covariance"]["data"],dtype=float)
        return Cov

In [108]:
pha = Asteroid('2008JL3')
pha.get_covariance()

array([[ 3.02e-07, -5.78e-09,  2.24e-06, -3.28e-07,  4.21e-07,  3.55e-07],
       [-5.78e-09,  1.12e-10, -4.27e-08,  6.26e-09, -7.73e-09, -6.81e-09],
       [ 2.24e-06, -4.27e-08,  1.67e-05, -2.44e-06,  3.21e-06,  2.63e-06],
       [-3.28e-07,  6.26e-09, -2.44e-06,  3.62e-07, -4.73e-07, -3.85e-07],
       [ 4.21e-07, -7.73e-09,  3.21e-06, -4.73e-07,  7.11e-07,  4.93e-07],
       [ 3.55e-07, -6.81e-09,  2.63e-06, -3.85e-07,  4.93e-07,  4.18e-07]])

# Prueba de set_date

In [131]:
class Asteroid:
    def __init__(self,ID, date = '2000-01-01 00:00:00'):
        self.ID = ID
        self.date = date
        if type(ID) is not str:
            raise ValueError(f"ID should be a string, specifically an asteroid name (EX: 2021EU), you passed {self.ID}")
        
    def set_date(self, date):
        self.date = date
        if type(date) is not str:
            raise ValueError(f"date should be a string as YYYY-MM-DD HH:MM:SS , you passed {date}")
    
    def get_date(self):
        return self.date

In [133]:
pha = Asteroid('2021EU')
pha.get_date()

'2000-01-01 00:00:00'

In [135]:
pha.set_date('2002-10-02 00:00:00')
pha.get_date()

'2002-10-02 00:00:00'

# Juntando todo

In [3]:
class Asteroid:
    """ Creates an object given an asteroid name.
    
        Parameter: 
            ID (str): Name of the Asteroid in standard notation.
            
        Example: 
            Ast = Asteroid('2008JL3')"""
    
    def __init__(self, ID):
        
        """Once you give an asteroide name, it will take the epoch data 
        from the JPL Small-Body Database as the default date parameter"""
        
        self.ID = ID
        if type(ID) is not str:
            raise ValueError(f"ID should be a string, specifically an asteroid name (EX: 2021EU), you passed {self.ID}")
     
        html = request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
        json_data = json.loads(html.read().decode())
        t0 = float(json_data["orbit"]["epoch"])
        self.epoch = Time(t0, format='jd', scale='utc').iso
        #We assume date equal as epoch
        self.date = self.epoch
        
        
    #GET DATE
    def get_date(self):
        #Para darle utilidad al método es preferible agregarle un return con el formato que el usuario pida
        """Useful for checking the ephemeris time 
        before calculating the surrogates.
        
        Returns:
                Date in UTC format (str)"""
        
        return self.date
    
    
    #SET DATE
    def set_date(self, date):
        
        """Default parameter: Asteroid's epoch. 
        Date should be a string in the UTC format.
        
        Parameter:
            date (str): Date on which you want to calculate the surrogates
                        in UTC format.
                        
        Example: 
            Asteroid.set_date('2020-01-01 00:00:00')
        
        
        WARNING: The date also should be later than the asteroid's epoch. 
        Dates before the epoch will be useless."""
        
        self.date = date
        if type(date) is not str:
            raise ValueError(f"date should be a string as YYYY-MM-DD HH:MM:SS , you passed {date}")
        
     
    #GET COVARIANCE
    def get_covariance(self): 
        
        """Returns:
                Covariance matrix of the asteroid. (Numpy Array)"""
        
        
        html = request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
        json_data = json.loads(html.read().decode())
        
        Cov = np.array(json_data["orbit"]["covariance"]["data"], dtype=float)
        return Cov
    
        
    #CALCULATE SURROGATES POSITIONS
    def calculate_surrogates(self, Nsur=1):
        
        """Given a quantity of surrogates you get the orbital state vector of each surrogate
        
        Parameter: 
                Nsur(int): Quantity of surrogates to be calculated. 
                            Default = 1
                            
        Returns: 
                np.array([x, y, z, vx, vy, vz]): Orbital State Vector (units = AU, AU/d)"""
    
        html=request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
        json_data=json.loads(html.read().decode())
    
        rad = 180/np.pi
        deg = 1/rad
        AU = 149597870.693 #km 
        mu = 132712440023.310 #km^3/s^2
    
        Cov = np.array(json_data["orbit"]["covariance"]["data"], dtype=float)
        Cov_label = json_data["orbit"]["covariance"]["labels"]
        t = float(json_data["orbit"]["epoch"])
        nlen = len(json_data["orbit"]["elements"])
    
        elnames = []
        elements = odict()
        
        for i in range(nlen):
            element = json_data["orbit"]["elements"][i]
            elements[element["name"]] = odict()
            for prop in element.keys():
                try:
                    elements[element["name"]][prop] = float(element[prop])
                except:
                    pass

        for elname in elements.keys():
            element = elements[elname]
            means = [elements['e']['value'], elements['q']['value'], elements['tp']['value'],
                     elements['om']['value'], elements['w']['value'], elements['i']['value']]
        
        data = np.random.multivariate_normal(means, Cov, Nsur, check_valid='ignore')
    
    
        e = data[:,0]; q = data[:,1]; tp = data[:,2]
        node = data[:,3]; peri = data[:,4]; inc = data[:,5]

        t0 = float(json_data["orbit"]["epoch"])
        et0 = spy.unitim(t0, "JDTDB", "ET")

        a = (q/(1 - e))*AU
        n = np.sqrt(mu/a**3)
        tps = np.array([spy.unitim(t, "JDTDB", "ET") for t in tp])
        M = n*(et0 - tps)
        Ms = np.mod(M, 2*np.pi)

        E_date = f'{self.date} UTC'
        et = spy.str2et(E_date)
        
        if et < et0:
            raise ValueError(f"Date must be higher than asteroid's epoch (epoch = {Time(t0, format='jd',scale='utc').iso})")
    
        Ast = []
        for i in range(len(e)): 
            Ast.append(spy.conics([q[i]*AU, e[i], inc[i]*deg, node[i]*deg, peri[i]*deg, Ms[i], et0, mu], et)/AU)
            Ast[i][3::] = Ast[i][3::]*86400
        Ast = np.array(Ast)
        return Ast

In [49]:
#pha=Asteroid('2021EU')
#pha.get_covariance(), pha.get_date()

In [50]:
#pha.set_date('2022-07-30 00:00:00')
#pha.calculate_surrogates()
#help(Asteroid)

# Nueva Clase: Body

In [26]:
class Body:
    
    """ Creates a celestial body as an object,
        using some methods from the Asteroid class"""
    
    def __init__(self, name, bodytype):
        
        """ Given a body name an a bodytype creates an object.
            Parameters:
                name (str): Name of the celestial body
                bodytype (int): Type of the body 
                                planet = 1
                                asteroid = 2
                                others = 3"""
        
        self.name = name
        self.bodytype = bodytype
        if type(bodytype) is not int:
            raise ValueError("Bodytype should be an integer. Planet = 1, Asteroid = 2, Others = 3")
        
    def get_jpl_positions(self,date):
        
        """ Given a date (in UTC format) it downloads from
            the Jet Propulsion Laboratory API the Orbital State
            Vector. 
            Parameter:
                date(str): Date in UTC format.
                
            Returns: 
                np.array([x, y, z, vx, vy, vz]): Orbital State Vector (units = AU, AU/d)"""
    
        if self.bodytype == 1:  
            JPL = Horizons(id=f'{self.name} Barycenter', 
                           location='@sun', 
                           epochs=Time(self.date).jd).vectors()
    
            rt = np.array([JPL[c].value[0] for c in ["x", "y", "z", "vx", "vy", "vz"]])
           
        if self.bodytype == 2:
            html = request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.name}&cov=mat")
            json_data = json.loads(html.read().decode())
            spkid = json_data['object']['spkid']
            JPL = Horizons(id=f'DES={spkid}', 
                           location='@sun', 
                           epochs=Time(date).jd).vectors()
    
            rt = np.array([JPL[c].value[0] for c in ["x", "y", "z", "vx", "vy", "vz"]])
     
        if self.bodytype == 3:
            JPL = Horizons(id=f'{self.name}', 
                           location='@sun', 
                           epochs=Time(date).jd).vectors()
    
            rt = np.array([JPL[c].value[0] for c in ["x", "y", "z", "vx", "vy", "vz"]])
        
        return rt
    

In [44]:
class BodyJPL:
    
    """ Creates a celestial body as an object,
        using some methods from the Asteroid class"""
    
    def __init__(self, name, bodytype):
        
        """ Given a body name an a bodytype creates an object.
            Parameters:
                name (str): Name of the celestial body
                bodytype (int): Type of the body 
                                planet = 1
                                asteroid = 2
                                others = 3"""
        
        self.name = name
        self.bodytype = bodytype
        if type(bodytype) is not int:
            raise ValueError("Bodytype should be an integer. Planet = 1, Asteroid = 2, Others = 3")
        
    def get_jpl_positions(self,date):
        
        """ Given a date (in UTC format) it downloads from
            the Jet Propulsion Laboratory API the Orbital State
            Vector. 
            Parameter:
                date(str): Date in UTC format.
                
            Returns: 
                np.array([x, y, z, vx, vy, vz]): Orbital State Vector (units = AU, AU/d)"""
        
        if self.bodytype == 1:  
            JPL = Horizons(id=f'{self.ID} Barycenter', 
                           location='@sun', 
                           epochs=Time(self.date).jd).vectors()
    
            rt = np.array([JPL[c].value[0] for c in ["x", "y", "z", "vx", "vy", "vz"]])
           
        if self.bodytype == 2:
            html = request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
            json_data = json.loads(html.read().decode())
            spkid = json_data['object']['spkid']
            JPL = Horizons(id=f'DES={spkid}', 
                           location='@sun', 
                           epochs=Time(self.date).jd).vectors()
    
            rt = np.array([JPL[c].value[0] for c in ["x", "y", "z", "vx", "vy", "vz"]])
     
        if self.bodytype == 3:
            JPL = Horizons(id=f'{self.ID}', 
                           location='@sun', 
                           epochs=Time(self.date).jd).vectors()
    
            rt = np.array([JPL[c].value[0] for c in ["x", "y", "z", "vx", "vy", "vz"]])
        
        return rt
    

In [47]:
class Asteroid(BodyJPL):
    """ Creates an object given an asteroid name.
    
        Parameter: 
            ID (str): Name of the Asteroid in standard notation.
            
        Example: 
            Ast = Asteroid('2008JL3')"""
    
    def __init__(self, ID):
        
        """Once you give an asteroide name, it will take the epoch data 
        from the JPL Small-Body Database as the default date parameter"""
        
        self.ID = ID
        if type(ID) is not str:
            raise ValueError(f"ID should be a string, specifically an asteroid name (EX: 2021EU), you passed {self.ID}")
     
        html = request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
        json_data = json.loads(html.read().decode())
        t0 = float(json_data["orbit"]["epoch"])
        self.epoch = Time(t0, format='jd', scale='utc').iso
        #We assume date equal as epoch
        self.date = self.epoch
        
        
    #GET DATE
    def get_date(self):
        #Para darle utilidad al método es preferible agregarle un return con el formato que el usuario pida
        """Useful for checking the ephemeris time 
        before calculating the surrogates.
        
        Returns:
                Date in UTC format (str)"""
        
        return self.date
    
    
    #SET DATE
    def set_date(self, date):
        
        """Default parameter: Asteroid's epoch. 
        Date should be a string in the UTC format.
        
        Parameter:
            date (str): Date on which you want to calculate the surrogates
                        in UTC format.
                        
        Example: 
            Asteroid.set_date('2020-01-01 00:00:00')
        
        
        WARNING: The date also should be later than the asteroid's epoch. 
        Dates before the epoch will be useless."""
        
        self.date = date
        if type(date) is not str:
            raise ValueError(f"date should be a string as YYYY-MM-DD HH:MM:SS , you passed {date}")
        
     
    #GET COVARIANCE
    def get_covariance(self): 
        
        """Returns:
                Covariance matrix of the asteroid. (Numpy Array)"""
        
        
        html = request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
        json_data = json.loads(html.read().decode())
        
        Cov = np.array(json_data["orbit"]["covariance"]["data"], dtype=float)
        return Cov
    
        
    #CALCULATE SURROGATES POSITIONS
    def calculate_surrogates(self, Nsur=1):
        
        """Given a quantity of surrogates you get the orbital state vector of each surrogate
        
        Parameter: 
                Nsur(int): Quantity of surrogates to be calculated. 
                            Default = 1
                            
        Returns: 
                np.array([x, y, z, vx, vy, vz]): Orbital State Vector (units = AU, AU/d)"""
    
        html=request.urlopen(f"https://ssd-api.jpl.nasa.gov/sbdb.api?sstr={self.ID}&cov=mat")
        json_data=json.loads(html.read().decode())
    
        rad = 180/np.pi
        deg = 1/rad
        AU = 149597870.693 #km 
        mu = 132712440023.310 #km^3/s^2
    
        Cov = np.array(json_data["orbit"]["covariance"]["data"], dtype=float)
        Cov_label = json_data["orbit"]["covariance"]["labels"]
        t = float(json_data["orbit"]["epoch"])
        nlen = len(json_data["orbit"]["elements"])
    
        elnames = []
        elements = odict()
        
        for i in range(nlen):
            element = json_data["orbit"]["elements"][i]
            elements[element["name"]] = odict()
            for prop in element.keys():
                try:
                    elements[element["name"]][prop] = float(element[prop])
                except:
                    pass

        for elname in elements.keys():
            element = elements[elname]
            means = [elements['e']['value'], elements['q']['value'], elements['tp']['value'],
                     elements['om']['value'], elements['w']['value'], elements['i']['value']]
        
        data = np.random.multivariate_normal(means, Cov, Nsur, check_valid='ignore')
    
    
        e = data[:,0]; q = data[:,1]; tp = data[:,2]
        node = data[:,3]; peri = data[:,4]; inc = data[:,5]
        
        spy.furnsh('naif0012.tls')
        t0 = float(json_data["orbit"]["epoch"])
        et0 = spy.unitim(t0, "JDTDB", "ET")

        a = (q/(1 - e))*AU
        n = np.sqrt(mu/a**3)
        tps = np.array([spy.unitim(t, "JDTDB", "ET") for t in tp])
        M = n*(et0 - tps)
        Ms = np.mod(M, 2*np.pi)

        E_date = f'{self.date} UTC'
        et = spy.str2et(E_date)
        
        if et < et0:
            raise ValueError(f"Date must be higher than asteroid's epoch (epoch = {Time(t0, format='jd',scale='utc').iso})")
    
        Ast = []
        for i in range(len(e)): 
            Ast.append(spy.conics([q[i]*AU, e[i], inc[i]*deg, node[i]*deg, peri[i]*deg, Ms[i], et0, mu], et)/AU)
            Ast[i][3::] = Ast[i][3::]*86400
        Ast = np.array(Ast)
        return Ast
    
    def compare_positions(self):
        self.bodytype = 2
        
        return super().get_jpl_positions(self)
        

In [48]:
Ast = Asteroid('2021eu')
Ast.compare_positions()

array([-1.02735744,  0.30252353,  0.00719919, -0.01652413, -0.01164168,
        0.00114186])

In [50]:
Ast.calculate_surrogates(Nsur=1)

array([[-1.0291892 ,  0.2953049 ,  0.00750515, -0.01644435, -0.01175516,
         0.00114568]])